In [1]:
import tensorflow as tf

In [2]:
# define
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [3]:
# menentukan input dan output
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, name="y")

In [4]:
# dnn atau hidden layer
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu, reuse=tf.AUTO_REUSE)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu, reuse=tf.AUTO_REUSE)
    
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs", reuse=tf.AUTO_REUSE)
    y_proba = tf.nn.softmax(logits)

In [5]:
# cost funtion cross entropy
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    loss_summary = tf.summary.scalar('log_loss', loss)

In [6]:
# training optimizer
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    training_op = optimizer.minimize(loss)

In [7]:
# fungsi eval
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)

In [8]:
# initialize semua variable
init = tf.global_variables_initializer()

# saver
saver = tf.train.Saver()

In [9]:
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [10]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X)
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [11]:
# training
n_epochs = 20
n_batches = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, n_batches):
            sess.run(training_op, feed_dict={X: X_batch, y:y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y:y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y:y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)
        
    save_path = saver.save(sess, './my_model.ckpt')

0 Batch accuracy: 1.0 Validation accuracy: 0.9644
1 Batch accuracy: 1.0 Validation accuracy: 0.979
2 Batch accuracy: 1.0 Validation accuracy: 0.9768
3 Batch accuracy: 1.0 Validation accuracy: 0.9766
4 Batch accuracy: 1.0 Validation accuracy: 0.9776
5 Batch accuracy: 1.0 Validation accuracy: 0.9744
6 Batch accuracy: 1.0 Validation accuracy: 0.9812
7 Batch accuracy: 1.0 Validation accuracy: 0.9804
8 Batch accuracy: 1.0 Validation accuracy: 0.9776
9 Batch accuracy: 1.0 Validation accuracy: 0.9824
10 Batch accuracy: 1.0 Validation accuracy: 0.9768
11 Batch accuracy: 1.0 Validation accuracy: 0.9784
12 Batch accuracy: 1.0 Validation accuracy: 0.984
13 Batch accuracy: 1.0 Validation accuracy: 0.9812
14 Batch accuracy: 1.0 Validation accuracy: 0.985
15 Batch accuracy: 1.0 Validation accuracy: 0.9822
16 Batch accuracy: 1.0 Validation accuracy: 0.9822
17 Batch accuracy: 1.0 Validation accuracy: 0.9834
18 Batch accuracy: 1.0 Validation accuracy: 0.983
19 Batch accuracy: 1.0 Validation accuracy: 0

In [17]:
# test
model_path = './my_model.ckpt'

with tf.Session() as sess:
    saver.restore(sess, model_path)
    accuracy_test = accuracy.eval(feed_dict={X: X_test, y: y_test})

INFO:tensorflow:Restoring parameters from ./my_model.ckpt


In [18]:
accuracy_test

0.9824